
# Exercise Notebook: Implementing RAG (Retrieval-Augmented Generation)

In this exercise notebook, you will go through the steps required to implement Retrieval-Augmented Generation (RAG).
The notebook will guide you through each step, providing explanations and asking you to fill in the code.

Please fill in the code cells where prompted to complete the implementation.

**Let's get started!**



## Installing Required Libraries

Before starting, ensure you have all the necessary libraries installed.
Install the following libraries by running the appropriate command below.

- `langchain`
- `langchain_community`
- `unstructured`
- `sentence_transformers`
- `tiktoken`
- `chromadb`
- `langchain_chroma`
- `langchain_groq`

Fill in the installation command in the code cell below:


In [1]:
!pip install langchain langchain_community unstructured sentence_transformers tiktoken chromadb langchain_chroma langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


## Import Necessary Modules

Now, you need to import the necessary modules to build the RAG system.
Write the import statements for the libraries required in the following code cell.


In [2]:
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
!unzip /content/IMDbMovies-Clean.csv.zip

Archive:  /content/IMDbMovies-Clean.csv.zip
  inflating: IMDbMovies-Clean.csv    


# Data Pre-processing and Preparation

In this section, we will focus on preparing the dataset for retrieval-based models. The steps involve cleaning the text, tokenizing it, and vectorizing it for further use in our model. These steps are essential for efficient retrieval and generation.

In [4]:
df = pd.read_csv('/content/IMDbMovies-Clean.csv')
df.head()

,Title,Summary,Director,Writer,Main Genres,Motion Picture Rating,Release Year,Runtime (Minutes),Rating (Out of 10),Number of Ratings (in thousands),Budget (in millions),Gross in US & Canada (in millions),Gross worldwide (in millions),Opening Weekend in US & Canada,Gross Opening Weekend (in millions)
0,Napoleon,An epic that details the checkered rise and fa...,Ridley Scott,David Scarpa,"Action,Adventure,Biography",R,2023.0,158.0,6.7,38.0,NaN,37.514,84.968,11.26.2023,20.639
1,The Hunger Games: The Ballad of Songbirds & Sn...,Coriolanus Snow mentors and develops feelings ...,Francis Lawrence,"Michael Lesslie,Michael Arndt,Suzanne Collins","Action,Adventure,Drama",PG-13,2023.0,157.0,7.2,37.0,100.0,105.043,191.729,11.19.2023,44.607
2,The Killer,"After a fateful near-miss, an assassin battles...",David Fincher,"Andrew Kevin Walker,Luc Jacamon,Alexis Nolent","Action,Adventure,Crime",R,2023.0,118.0,6.8,117.0,NaN,NaN,0.421,NaN,NaN
3,Leo,A 74-year-old lizard named Leo and his turtle ...,"David Wachtenheim,Robert Smigel,Robert Marianetti","Paul Sado,Robert Smigel,Adam Sandler","Animation,Comedy,Family",PG,2023.0,102.0,7.0,10.0,NaN,NaN,NaN,NaN,NaN
4,Thanksgiving,"After a Black Friday riot ends in tragedy, a m...",Eli Roth,"Eli Roth,Jeff Rendell","Horror,Mystery,Thriller",R,2023.0,106.0,7.0,9.1,NaN,25.409,29.667,11.19.2023,10.306


# Read Files from the Directory

In this step, we will read all text-based files from a specified directory. The files could be in various formats such as Markdown (`.md`), plain text (`.txt`), or other similar formats. We will handle each file based on its extension and process it accordingly.

### Steps to Follow:

1. **Specify the directory**: Define the directory from which to load the files.
2. **Read files by extension**: Filter files based on their extensions (e.g., `.md`, `.txt`, etc.).
3. **Convert or process content**: For each file, load the content. For markdown files, we will convert them into HTML using the `markdown` module. For other text formats, we will simply read the content as plain text.
4. **Store the processed content**: The result of each file’s content will be stored in a list for further use.

In [5]:
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [7]:
for i in range (0,5_000):
  title = df['Title'].iloc[i]
  content = df['Summary'].iloc[i]

  markdown_content = f"# {title}\n\n"
  markdown_content += f"{content}\n\n"
  with open(f'{directory}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)

In [8]:
markdown_texts = []
for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_texts.append(html_content)

## Split the Text into Chunks

In this step, we will split the text into manageable chunks. This is important for tasks such as document retrieval and text generation, where large bodies of text need to be broken down for efficient processing.

### Why Split Text into Chunks?

- **Memory Efficiency**: Working with smaller pieces of text is more memory efficient.
- **Improved Retrieval**: Splitting long documents into smaller sections can improve the relevance of retrieval tasks.
- **Better Generation**: For text generation, smaller chunks help models focus on a specific context.

### Steps to Follow:

1. **Specify the chunk size**: Define the maximum number of words or characters per chunk.
2. **Split the text**: Split each document or file content into chunks based on the defined size.
3. **Handle incomplete chunks**: If a document ends with a chunk that is smaller than the chunk size, include it as a valid chunk.
4. **Store the chunks**: Store all chunks in a list for further processing.

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.create_documents(markdown_texts)

## Initialize the Embedding Model & Create a Vector Store Using Chroma

In this step, we will initialize an embedding model to convert text chunks into numerical vectors. These embeddings will be used to measure the similarity between different chunks of text. After generating the embeddings, we will store them using Chroma, a vector store designed to efficiently manage and retrieve embeddings.

### Steps to Follow:

1. **Initialize the embedding model**: Choose an embedding model (e.g., Sentence Transformers or OpenAI embeddings) to convert text into vectors.
2. **Generate embeddings**: Convert each text chunk into its corresponding embedding.
3. **Create a vector store**: Use Chroma to store the embeddings and their associated metadata (e.g., the original text chunk).
4. **Verify the store**: Ensure that the embeddings are stored correctly and that you can retrieve them based on similarity.

In [10]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(documents, embedding_function, persist_directory="./chroma_db")

<ipython-input-10-54f91d5a9530>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://h

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Load the Persistent Directory for Chroma DB

In this step, we will focus on **loading** the persistent storage for Chroma DB. This allows us to access previously stored embeddings and metadata without recomputing them. By setting up persistent storage, we ensure that the vector database can be saved to disk and loaded again when needed.

### Steps to Follow:

1. **Specify the persistent directory**: Identify the directory where the Chroma DB is stored.
2. **Load the vector store**: Use Chroma to load the embeddings and metadata from this directory.
3. **Verify the loaded data**: Ensure that the embeddings and associated data have been correctly loaded and can be queried.

In [11]:
import os
import json
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [12]:
def query_chroma_db(query, db, top_k=7):
    docs=db.similarity_search(query)
    result=[doc.page_content for doc in docs]
    return result

In [26]:
query_chroma_db("The number of movies", db)

['<h1>Movie 43</h1>\n<p>A series of interconnected short films follows a washed-up producer as he pitches insane story lines featuring some of the biggest stars in Hollywood.</p>',
 '<h1>8½</h1>\n<p>A harried movie director retreats into his memories and fantasies.</p>',
 '<h1>King Kong</h1>\n<p>A greedy film producer assembles a team of moviemakers and sets out for the infamous Skull Island, where they find more than just cannibalistic natives.</p>',
 '<h1>Sunset Blvd.</h1>\n<p>A screenwriter develops a dangerous relationship with a faded film star determined to make a triumphant return.</p>']

In [14]:
prompt_template = """
Answer the question based only on the following context:
Context: {context}
Question: {question}
Your answer:
"""

prompt_template = PromptTemplate(
    template=prompt_template, input_varibales=["context","question"]
)

# Create & Test the Retrieval with a Sample Query

In this step, we will set up the retrieval process using the embeddings stored in Chroma DB. Retrieval is a key part of the Retrieval-Augmented Generation (RAG) pipeline, allowing us to find relevant documents or text chunks based on a query. After setting up the retrieval system, we will test it with a sample query to ensure that it returns the most relevant chunks.

### Steps to Follow:

1. **Set up the retrieval system**: Using the Chroma DB with the stored embeddings, create a retrieval function that can match a query to relevant text chunks.
2. **Prepare a sample query**: Define a query that you want to search for in the stored text chunks.
3. **Retrieve relevant chunks**: Use the query to search the vector store and retrieve the most similar chunks.
4. **Test the results**: Check that the returned chunks are relevant to the query and adjust the retrieval system if needed.

In [15]:
groq_api_key = "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"
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

In [16]:
MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

<ipython-input-16-a2932d0d6a9b>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  MODEL = LLMChain(llm=llm,


In [17]:
def query_rag(query:str):
  similarty_serch=db.similarity_search(query, k=4)
  context_text="\n\n".join([doc.page_content for doc in similarty_serch])
  rag_response=MODEL.invoke({"context":context_text, "question":query})
  return rag_response

In [29]:
response = query_rag("Give a movie about a princess fighting an evil")
response

Prompt after formatting:

Answer the question based only on the following context:
Context: <h1>Cinderella</h1>
<p>A modern movie musical with a bold take on the classic fairy tale. Our ambitious heroine has big dreams and with the help of her fab Godmother, she perseveres to make them come true.</p>

<h1>The Princess</h1>
<p>When a strong-willed princess refuses to wed the cruel sociopath, she is kidnapped and locked in a remote tower of her father’s castle. With her vindictive suitor intent on taking her father’s throne, the princess must save the kingdom.</p>

<h1>Epic Movie</h1>
<p>A spoof on previous years' epic movies (Da Vinchi Shifresi (2006), The Chronicles of Narnia: The Lion, the Witch and the Wardrobe (2005) + 20 more), TV series, music videos and celebs. 4 orphans are on an epic adventure.</p>

<h1>Snow White and the Evil Queen</h1>
<p>A story about beauty and vanity, a princess and a prince, and about love and its power to raise from death to life.</p>
Question: Give a mo

{'context': "<h1>Cinderella</h1>\n<p>A modern movie musical with a bold take on the classic fairy tale. Our ambitious heroine has big dreams and with the help of her fab Godmother, she perseveres to make them come true.</p>\n\n<h1>The Princess</h1>\n<p>When a strong-willed princess refuses to wed the cruel sociopath, she is kidnapped and locked in a remote tower of her father’s castle. With her vindictive suitor intent on taking her father’s throne, the princess must save the kingdom.</p>\n\n<h1>Epic Movie</h1>\n<p>A spoof on previous years' epic movies (Da Vinchi Shifresi (2006), The Chronicles of Narnia: The Lion, the Witch and the Wardrobe (2005) + 20 more), TV series, music videos and celebs. 4 orphans are on an epic adventure.</p>\n\n<h1>Snow White and the Evil Queen</h1>\n<p>A story about beauty and vanity, a princess and a prince, and about love and its power to raise from death to life.</p>",
 'question': 'Give a movie about a princess fighting an evil',
 'text': 'The Princess'

In [30]:
print(f'Context:\n{response["context"]}\n\nQuestion:\n{response["question"]}\n\nText: \n{response["text"]}')

Context:
<h1>Cinderella</h1>
<p>A modern movie musical with a bold take on the classic fairy tale. Our ambitious heroine has big dreams and with the help of her fab Godmother, she perseveres to make them come true.</p>

<h1>The Princess</h1>
<p>When a strong-willed princess refuses to wed the cruel sociopath, she is kidnapped and locked in a remote tower of her father’s castle. With her vindictive suitor intent on taking her father’s throne, the princess must save the kingdom.</p>

<h1>Epic Movie</h1>
<p>A spoof on previous years' epic movies (Da Vinchi Shifresi (2006), The Chronicles of Narnia: The Lion, the Witch and the Wardrobe (2005) + 20 more), TV series, music videos and celebs. 4 orphans are on an epic adventure.</p>

<h1>Snow White and the Evil Queen</h1>
<p>A story about beauty and vanity, a princess and a prince, and about love and its power to raise from death to life.</p>

Question:
Give a movie about a princess fighting an evil

Text: 
The Princess


In [25]:
def query_rag_with_threshold(query: str, threshold: float):
    similarity_search_results = db.similarity_search_with_score(query, k=4)
    context_text = "\n\n".join([doc.page_content for doc, score in similarity_search_results if score > threshold])
    rag_response = MODEL.invoke({"context": context_text, "question": query})
    return rag_response